In [2]:
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [28]:
import math
import numpy as np
class Point(np.ndarray):
    def __new__(cls, *args, **kwargs):    
        return super(Point, cls).__new__(cls, shape = (len(args), ), **kwargs)
    
    def __init__(self, *args):
        self[:] = args

    @property
    def dims(self):
        return len(self)
    
    def distance(self, other):
        assert self.dims == other.dims, f'Points have different dimensions: {self.dims} and {other.dims}'
        return (self - other).length()
    
    @property
    def length(self):
        return math.sqrt((self*self).sum())
Vector = Point # dirty hack

# class Vector(np.ndarray):
#     def __new__(cls, *args, **kwargs):    
#         return super(Point, cls).__new__(cls, shape = (len(args), ), **kwargs)
    
#     def __init__(self, *args):
#         print(args)
#         self[:] = args

#     @property
#     def dims(self):
#         return len(self)
p1 = Point(1,2)
p2 = Point(3,4)
type(p1 + p2)

__main__.Point

In [ ]:
import random 
#random.random()	Returns a random float number between 0 and 1
# random.uniform(10.5, 75.5)	Returns a random float number between a range
# round(random.uniform(33.33, 66.66), 2)	Returns a random float number up to 2 decimal places
# random.SystemRandom().uniform(5, 10)	Returns a secure random float number
# numpy.random.uniform()	Returns a random array of floats

In [30]:
from dataclasses import dataclass
from enum import Enum

class Shape(Enum):
    Box = 1
    Sphere = 2
    Planet = 3
    MobiusBox = 4

REQUIRED_DIMENSIONS = {
    Shape.Box: {'length', 'width',  'height'}
}

    
DEFAULT_T = 1
DEFAULT_M = 1

class Particle:
    # coordinates
    coordinates: Point
    # temperature
    T: float #todo: rename to temperature
    # mass
    m: float # todo: rename to mass
    
    
    # Pressure
    
    # calculated: total pressure from all neighbours
    # Volume
    # T * m / P
    def volume(self):
        return self.T * self.m / self.pressure
    
    def __init__(self, x, y, z, T=DEFAULT_T, m=DEFAULT_M):
        self.coordinates = Point(x,y,z)
        self.T = T
        self.m = m
SUPPORTED_SHAPES = {
    Shape.Box
}    
class Simulation:
    # Space borders
    def __init__(num_particles: int, shape: Shape, dimensions: dict):
        if shape not in SUPPORTED_SHAPES:
            raise NotImplemented("The shape {shape} is not yet supported")
        # shape
        self.shape = shape
        # check dimensions
        if not set(dimensions.keys()) == REQUIRED_DIMENSIONS[shape]:
            raise ValueError(f"Shape {shape} requires following dimensions: {REQUIRED_DIMENSIONS[shape]}")
        
        self.particles = []
        # generate particles
        if shape == Shape.Box:
            # borders: 6 planes with equations
            # x = 0, x = self.lenght
            # y = 0, y = self.width
            # z = 0, z = self.height
            self.length = dimensions['length']
            self.width = dimensions['width']
            self.height = dimensions['height']
        
            # random
            for _ in range(num_particles):
                x = random.random() * self.length
                y = random.random() * self.width
                z = random.random() * self.height
                particle = Particle(x,y,z,DEFAULT_T, DEFAULT_M)
                self.particles.append(particle)
            
        
    
    # interactive simulation with forward-moving time
    def update_positions(self, dt):
        for particle in self.particles:
            # calculate forces between particles
            total_force = Vector(0,0,0)
            pressure = 0
            for other_particle in self.particles:
                if other_particle is particle:
                    continue
                
                # direction = self.position - other.position
                # value = T / distance
                diff = particle.coordinates - other_particle.coordinates
                force = (particle.T + other_particle.T) * diff / diff.length **2
                total_force += force
                total_pressure += force.length
                

            # update positions 

            # check border collisions, bounce off the borders.
            pass
    

In [ ]:
pygame.draw.circle

In [32]:

    
import pygame
import math
pygame.init()

WIDTH, HEIGHT =  800, 800
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Planet Simulation")

WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)
BLUE = (100, 149, 237)
RED = (188, 39, 50)
DARK_GREY = (80, 78, 81)

FONT = pygame.font.SysFont("comicsans", 16)

class Planet:
	AU = 149.6e6 * 1000
	G = 6.67428e-11
	SCALE = 250 / AU  # 1AU = 100 pixels
	TIMESTEP = 3600*24 # 1 day

	def __init__(self, x, y, radius, color, mass):
		self.x = x
		self.y = y
		self.radius = radius
		self.color = color
		self.mass = mass

		self.orbit = []
		self.sun = False
		self.distance_to_sun = 0

		self.x_vel = 0
		self.y_vel = 0

	def draw(self, win):
		x = self.x * self.SCALE + WIDTH / 2
		y = self.y * self.SCALE + HEIGHT / 2

		if len(self.orbit) > 2:
			updated_points = []
			for point in self.orbit:
				x, y = point
				x = x * self.SCALE + WIDTH / 2
				y = y * self.SCALE + HEIGHT / 2
				updated_points.append((x, y))

			pygame.draw.lines(win, self.color, False, updated_points, 2)

		pygame.draw.circle(win, self.color, (x, y), self.radius)
		
		if not self.sun:
			distance_text = FONT.render(f"{round(self.distance_to_sun/1000, 1)}km", 1, WHITE)
			win.blit(distance_text, (x - distance_text.get_width()/2, y - distance_text.get_height()/2))

	def attraction(self, other):
		other_x, other_y = other.x, other.y
		distance_x = other_x - self.x
		distance_y = other_y - self.y
		distance = math.sqrt(distance_x ** 2 + distance_y ** 2)

		if other.sun:
			self.distance_to_sun = distance

		force = self.G * self.mass * other.mass / distance**2
		theta = math.atan2(distance_y, distance_x)
		force_x = math.cos(theta) * force
		force_y = math.sin(theta) * force
		return force_x, force_y

	def update_position(self, planets):
		total_fx = total_fy = 0
		for planet in planets:
			if self == planet:
				continue

			fx, fy = self.attraction(planet)
			total_fx += fx
			total_fy += fy

		self.x_vel += total_fx / self.mass * self.TIMESTEP
		self.y_vel += total_fy / self.mass * self.TIMESTEP

		self.x += self.x_vel * self.TIMESTEP
		self.y += self.y_vel * self.TIMESTEP
		self.orbit.append((self.x, self.y))

        
DEFAULT_ZOOM = 1

class Camera:
    # position 
    center: Point
    # direction
    direction: Point # vector
    # todo: and also camera orientation! Which way is up.
    # zoom, e.g. camera angle
    zoom: float
    # todo: there's also a setting to prevent drawing objects too close to the camera or behind the camera.
    
    def __init__(self, center:tuple, direction:tuple, zoom = DEFAULT_ZOOM):
        self.center = Point(*center)
        self.direction = Point(*direction)
        self.direction /= self.direction.length()
        self.zoom = zoom

def draw(simulation: , camera, window, window_size):
    # draw all particles
        # size = Volume
        # color = Temperature
        # trail?
    for particle in simulation.particles:
        # flat projection
        # 
        point = particle.coordinates
        normale = point - camera.center
        projected_point = point - camera.direction * (camera.direction * (point - camera.center)).sum()
        
        # i now need to get the 2d coordinates of the point...
        # todo
        # for now let's assume the camera always points vertically up, and after projection 
        
        # if the point is in the field of view
        if camera.center.distance(projected_point) > zoom:
            color = WHITE
            # todo: calculate color based on temp. something like gradient.
            # color = round((RED * T + BLUE * (RED_TEMPERATURE-T))/RED_TEMPERATURE)
            x, y, z = point
            pygame.draw.circle(win, color, (x, y), self.radius)
            
        
    # draw borders
    if simulation.shape == Shape.Box:
        pass #todo
        
DEFAULT_NUM_PARTICLES = 100
DEFUALT_SHAPE = Shape.Box
DEFAULT_DIMENSIONS = {'lenght':0.5,'width':0.5,'height':0.5}

DEFAULT_CAMERA_CENTER = (0,0,0)
DEFAULT_CAMERA_DIRECTION = (0,0,1)

def main():
	run = True
	clock = pygame.time.Clock()
    
    simulation = Simulation(num_particles=DEFAULT_NUM_PARTICLES,shape=DEFUALT_SHAPE,dimensions=DEFAULT_DIMENSIONS)
    camera = Camera(DEFAULT_CAMERA_CENTER, DEFAULT_CAMERA_DIRECTION, DEFAULT_ZOOM)
    

	while run:
		clock.tick(60)
		WIN.fill((0, 0, 0))

		for event in pygame.event.get():
			if event.type == pygame.QUIT:
				run = False
                
            # update camera positions with mouse
            # todo
        
        simulation.update_positions(dt=SPEED)
        draw(simulation, camera, WIN, (WIDTH, HEIGHT))

		pygame.display.update()

	pygame.quit()


main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 135)

In [9]:
import numpy as np
x = np.array([1,2])
y = np.array([3,4])
x + y

array([4, 6])